In [10]:
# from classifier_lewidi import *
# from params_lewidi import *
from DataManager import *

import os
import argparse
from datetime import datetime
import pytz
import re


In [11]:


filepath1 = '../data/ConvAbuse_dataset/'
filepath2 = '../data/ArMIS_dataset/'
filepath3 = '../data/HS-Brexit_dataset/'
filepath4 = '../data/MD-Agreement_dataset/'




In [12]:

data = DataManager(filepath1, filepath2, filepath3, filepath4)

hello3
I'm in with the conv_train dataset


KeyboardInterrupt: 

In [24]:
print(data.dataset_groups)

defaultdict(<class 'list'>, {'conv_train':                                  text annotations      annotators  \
0                                 yes       1,1,1  Ann2,Ann3,Ann7   
1         were you not ten years old?       1,1,1  Ann3,Ann7,Ann8   
2                                 yea        -1,1       Ann1,Ann7   
3                              Ibadan         1,1       Ann3,Ann6   
4                           Bangalore       1,1,1  Ann1,Ann2,Ann3   
...                               ...         ...             ...   
2393  more worthless fucking bot spam    -2,-2,-2  Ann1,Ann4,Ann8   
2394                          fuck no     -1,-1,0  Ann1,Ann6,Ann7   
2395                             fuck    -1,-2,-1  Ann1,Ann2,Ann3   
2396                      yes it does       1,1,1  Ann4,Ann6,Ann7   
2397                      i dont know       1,1,1  Ann2,Ann3,Ann5   

      soft_label_0  soft_label_1 hard_label  
0            1.000         0.000          0  
1            1.000         0.000    

In [25]:
def_params = params_lewidi()

NameError: name 'params_lewidi' is not defined

In [ ]:

key = "conv_train"

annotators = list(sorted(set(itertools.chain.from_iterable(data[key]['annotators'].str.findall("\w+")))))
    
model = ToxicityClassifier(data, annotators=annotators, params=def_params)



In [ ]:
score, results = model.CV()

score["params"] = ", ".join(key + ": " + str(val) for key, val in def_params.__dict__.items())
score["task"] = model.params.task

pacific = pytz.timezone('US/Pacific')
sa_time = datetime.now(pacific)
name_time = sa_time.strftime('%m%d%y-%H:%M')
score["time"] = name_time
print(score)

score_dir = os.path.join(def_params.source_dir, "results", "GHC", "classification.csv")
result_dir = os.path.join(def_params.source_dir, "results", "GHC", name_time + "_" + def_params.task + ".csv")
if os.path.exists(score_dir):
    pd.DataFrame.from_records([score]).to_csv(score_dir, header=False,  index=False, mode="a")
else:
    pd.DataFrame.from_records([score]).to_csv(score_dir, index=False)

    pd.DataFrame.from_dict(results).to_csv(result_dir, index=False)
